In [17]:
import rosbag
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2

# Function to read point clouds from a ROS bag file
def read_point_clouds(bag_file, topic):
    point_clouds = []
    bag = rosbag.Bag(bag_file, 'r')
    for _, msg, _ in bag.read_messages(topics=[topic]):
        # Read the point cloud data
        cloud_points = pc2.read_points(msg, field_names=("x", "y", "z"), skip_nans=True)
        point_clouds.append(np.array(list(cloud_points)))
    bag.close()
    return point_clouds

# Replace with your bag file and topic
bag_file = '2023-10-15-16-26-34.bag'
topic = '/your_pointcloud_topic'
point_clouds = read_point_clouds(bag_file, topic)


ModuleNotFoundError: No module named 'rosbag'

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

def preprocess_point_cloud(point_cloud, num_points=2048):
    """
    Preprocess point cloud data for PointNet:
    - Randomly sample a fixed number of points.
    - Normalize the coordinates.
    """
    # Randomly sample points if there are too many
    if len(point_cloud) > num_points:
        idxs = np.random.choice(len(point_cloud), num_points, replace=False)
        sampled_points = point_cloud[idxs, :]
    else:
        sampled_points = point_cloud

    # Normalize the point cloud
    scaler = StandardScaler()
    normalized_points = scaler.fit_transform(sampled_points)

    return normalized_points

# Assuming 'point_clouds' is a list of numpy arrays with point cloud data
preprocessed_point_clouds = [preprocess_point_cloud(pc) for pc in point_clouds]

# Now 'preprocessed_point_clouds' can be used as input to PointNet


In [ ]:
!pip3 install rosbags


  Obtaining dependency information for rosbags from https://files.pythonhosted.org/packages/2f/f8/9139638c7e9a407cd9a109ac60a2e0521356437136efbd06b924d7521c62/rosbags-0.9.16-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 13.3 MB/s eta 0:00:00
  Obtaining dependency information for ruamel.yaml from https://files.pythonhosted.org/packages/57/db/4b74a29f5fd175aea3ff0d95f8230d9bb8a54e38d963c6f96065b9e2eef3/ruamel.yaml-0.18.5-py3-none-any.whl.metadata
  Obtaining dependency information for zstandard from https://files.pythonhosted.org/packages/54/fc/c1b1a1e140451f3362789f546731b3ef36c78668be19d7fc6fbd4326b535/zstandard-0.22.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for ruamel.yaml.clib>=0.2.7 from https://files.pythonhosted.org/packages/01/b0/4ddef56e9f703d7909febc3a421d709a3482cda25826816ec595b73e3847/ruamel.yaml.clib-0.2.8-cp311-cp311-macosx_13_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/

In [ ]:
import open3d as o3d
import rosbag
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
import numpy as np

def read_pointcloud2_from_rosbag(rosbag_path, topic):
    bag = rosbag.Bag(rosbag_path, 'r')
    for topic, msg, t in bag.read_messages(topics=[topic]):
        # Convert the PointCloud2 message to an array of points
        # The 'field_names' must correspond to the fields in your PointCloud2 message
        cloud_points = pc2.read_points(msg, field_names=("x", "y", "z"), skip_nans=True)
        points = np.array(list(cloud_points))
        yield points, t
    bag.close()

def voxelize_pointcloud(points, voxel_size):
    # Convert the numpy array of points to an Open3D point cloud
    o3d_pc = o3d.geometry.PointCloud()
    o3d_pc.points = o3d.utility.Vector3dVector(points)

    # Voxelization
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3d_pc, voxel_size)
    return voxel_grid

# Specify the path to your ROS bag and the point cloud topic
rosbag_path = 'path_to_your_rosbag.bag'
pointcloud_topic = '/your_pointcloud_topic'
voxel_size = 0.05  # for example, 5cm voxel size

# Process each point cloud
for points, t in read_pointcloud2_from_rosbag(rosbag_path, pointcloud_topic):
    voxel_grid = voxelize_pointcloud(points, voxel_size)
    # Now you can use the voxel_grid for further processing or as input to your model
    # For example, you can convert the voxel grid to a format suitable for your neural network


ModuleNotFoundError: No module named 'open3d'

In [1]:
!pip3 install open3d

In [4]:
import open3d as o3d
import numpy as np
import os

def load_point_cloud_from_txt(file_path):
    with open(file_path, 'r') as f:
        points = [list(map(float, line.strip().split())) for line in f]
    return np.array(points)

def voxelize_point_cloud(points, voxel_size):
    o3d_pc = o3d.geometry.PointCloud()
    o3d_pc.points = o3d.utility.Vector3dVector(points)
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3d_pc, voxel_size)
    return voxel_grid

def process_directory_of_point_clouds(directory_path, voxel_size):
    voxel_grids = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.txt'):  # Check for text files
            file_path = os.path.join(directory_path, file_name)
            points = load_point_cloud_from_txt(file_path)
            voxel_grid = voxelize_point_cloud(points, voxel_size)
            voxel_grids.append(voxel_grid)
            # Optionally, save each voxel grid to a file
            # o3d.io.write_voxel_grid(file_path.replace('.txt', '.ply'), voxel_grid)
    return voxel_grids

# Specify the directory containing the point cloud text files and voxel size
point_cloud_directory = 'rosbag_txt'
voxel_size = 0.05  # For example, 5cm voxel size

# Process all point cloud text files in the specified directory
voxel_grids = process_directory_of_point_clouds(point_cloud_directory, voxel_size)

# 'voxel_grids' now contains a list of voxel grid objects for each point cloud


ModuleNotFoundError: No module named 'open3d'